---
기본적인 학습 프로세스 돌아보기
1. 데이터 만들기
1. 모델 만들기
1. loss, optimization  (컴파일할 준비)
1. train 학습 진행
1. evaluation

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

## Hyperparameter Tunning
hyper parameter 는 주로 패키지 밑에 둔다

In [2]:
num_epochs = 1
batch_size = 64

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [3]:
# 데이터 불러오기
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

In [4]:
# 차원 늘리기
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

# normalization
train_x = train_x / 255.
test_x = test_x / 255.

## Build Model

In [5]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

## Optimization

In [6]:
# Optimization 
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training & Evaluating

In [7]:
# 학습 (Training)
model.fit(train_x, train_y,   
             batch_size=batch_size, 
             shuffle=True)

# evaluating
model.evaluate(test_x, test_y, batch_size=batch_size)

157/157 [==============================] - 3s 15ms/step - loss: 0.0542 - accuracy: 0.9827


[0.05419684946537018, 0.982699990272522]

---
# PyTorch

In [8]:
import torch

# 모델 설계를 위한 nn, F
import torch.nn as nn
import torch.nn.functional as F

# Optimization 을 
import torch.optim as optim

# torchvision 은 주로 이미지 데이터를 관리하기 위한 모듈
from torchvision import datasets, transforms

# ※ TensorFlow 의 dataset 은 예제를 불러오기 위함 이었지만,
# PyTorch 의 dateset 은 그 안에 DataLoader 를 만들어서 데이터를 불러오기 위함이다.

## Hyper Parameter

In [9]:
seed = 1   # PyTorch 는 랜덤하게 학습을 하다 보니까, 학습 랜덤값을 고정시키기 위해 seed 사용

lr = 0.001   # learning rate
momentum = 0.5  # lr 에 같이 넣어주는 param

batch_size = 64
test_batch_size = 64

epochs = 5  # 보통은 수만 ~ 수십만 epoch 돌린다

no_cuda = False    # CUDA 설치되어 있으면 True 로 하세요
log_interval = 100  # 학습 진행하면서 하나의 batch 안에서도 여러 step 이 진행되는데 몇 step 마다 로그 찍을지 세팅

## Model

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) #※ TF 에선 out channel 만 명시하면 되었다 
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)  # ※ TF 에선 dense 라고 불렀었죠
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [12]:
save_dir = r'D:\DevRoot\DataSet\torch\dataset'

In [13]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


# 얼핏 보았을때 이 부분은 TF 가 웬지 더 간편해 보인다.  (dataset 에서 불러와서 곧바로 fit 에 넣었으니..)
# 그러나 TF 는 준비된 샘플 데이터를 편하게 가져온것 뿐이고..
# 실제 데이터를 새로 받아서 처리 할때는 PyTorch 의 아래와 같은 형태가 더 강한 
# 장점들을 갖고 있다.  물론, 상황이나 데이터 형태에 따라 다를것이다.
# DataLoader 를 사용하여 데이터 추가하기도 좋고, 좀더 통일성 있고.
# 좀더 pythoninc 하구..

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(save_dir, train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)



test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(save_dir, train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

## Optimization

In [16]:
model = Net().to(device)  # 컴파일
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

# TF 는 model.compile() 안에서 optimization 을 넣고 loss 도 넣어줗었다.
# PyTorch 는 일단 컴파일만 하고 
# Optimizer 설정후 Loss 는 아래에서 학습하면서 들어간다

## Training & Evaluation

In [ ]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))